# HDF-N master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table
import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180427"
FIELD = "HDF-N"
catname = "../../dmu1/dmu1_ml_HDF-N/data/master_catalogue_hdf-n_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
r_bands = [           "GPC1 r", "SUPRIME r"]
i_bands = [           "GPC1 i", "SUPRIME i"]
z_bands = [           "GPC1 z", "SUPRIME z"]
K_bands = [                                 "WIRCAM K", "MOIRCS K"]

all_bands = [r_bands, i_bands, z_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry catalogue
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 6
  Total magnitude
    Number of flagged objects: 7
GPC1 r
  Aperture magnitude
    Number of flagged objects: 9
  Total magnitude
    Number of flagged objects: 9
GPC1 i
  Aperture magnitude
    Number of flagged objects: 7
  Total magnitude
    Number of flagged objects: 10
GPC1 z
  Aperture magnitude
    Number of flagged objects: 4
  Total magnitude
    Number of flagged objects: 5
GPC1 y
  Aperture magnitude
    Number of flagged objects: 2
  Total magnitude
    Number of flagged objects: 2


## 2. Outliers

In [6]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):
        #print(band1, band2)

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))          
                      

SUPRIME r (aperture) - GPC1 r (aperture):
  Number of outliers: 0
SUPRIME r (total) - GPC1 r (total):
  Number of outliers: 0
One of m_ap_gpc1_i and m_ap_suprime_i is not present in the catalogue.
One of m_gpc1_i and m_suprime_i is not present in the catalogue.
One of m_ap_gpc1_z and m_ap_suprime_z is not present in the catalogue.
One of m_gpc1_z and m_suprime_z is not present in the catalogue.
One of m_ap_wircam_k and m_ap_moircs_k is not present in the catalogue.
One of m_wircam_k and m_moircs_k is not present in the catalogue.


## 3. Save table

In [ ]:
#Merge any aperture flags
for col in master_catalogue.colnames:
    if col.startswith("flag_ap_"):
        try:
            master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
            master_catalogue.remove_column(col)
        except KeyError:
            print("{} only has aperture flags.".format(col))
            master_catalogue.rename_column(col, col.replace("_ap_", "_"))

In [7]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)